# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
0.983033,"""16422041086677…","""../../../../da…","""17000072905979…","""59288463577833…","""56862967937937…",false
0.144221,"""82589721783404…","""../../../../da…","""90559228707515…","""59288463577833…","""12657845020284…",true
0.679745,"""10576810793280…","""../../../../da…","""33680953094809…","""22757383136810…","""56862967937937…",false
1.0,"""63168250606752…","""../../../../da…","""90559228707515…","""18206536403758…","""56862967937937…",false
0.718982,"""13531780026445…","""../../../../da…","""17000072905979…","""18206536403758…","""10611334444899…",true
0.0,"""16054307055777…","""../../../../da…","""33680953094809…","""17416580503860…",null,false
0.02333,"""11848528886670…","""../../../../da…","""33680953094809…","""18206536403758…","""10611334444899…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""31530602218362…","""../../../../da…","""13760787222162…","""FirstMortgage3…","""11110435185100…","""Rejected"""
0.144221,"""75652576398783…","""../../../../da…","""71091621030751…","""FirstMortgage3…","""17494588542242…","""Accepted"""
0.679745,"""42897405751837…","""../../../../da…","""14400430783528…","""MoneyMarketSav…","""11110435185100…","""Rejected"""
1.0,"""91079531202445…","""../../../../da…","""71091621030751…","""BasicChecking""","""11110435185100…","""Rejected"""
0.718982,"""10858209551657…","""../../../../da…","""13760787222162…","""BasicChecking""","""69390031378793…","""Accepted"""
0.0,"""17330502091498…","""../../../../da…","""14400430783528…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""13846373102457…","""../../../../da…","""14400430783528…","""BasicChecking""","""69390031378793…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""38413520582630…","""../../../../da…","""47667404552092…","""FirstMortgage3…","""67937159366821…","""Rejected"""
0.144221,"""16201366745835…","""../../../../da…","""26266512149204…","""FirstMortgage3…","""73825661548594…","""Accepted"""
0.679745,"""89282738104599…","""../../../../da…","""24350440080405…","""MoneyMarketSav…","""67937159366821…","""Rejected"""
1.0,"""80071111162905…","""../../../../da…","""26266512149204…","""BasicChecking""","""67937159366821…","""Rejected"""
0.718982,"""16641135377085…","""../../../../da…","""47667404552092…","""BasicChecking""","""13648128567648…","""Accepted"""
0.0,"""58327523606789…","""../../../../da…","""24350440080405…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""80285002962964…","""../../../../da…","""24350440080405…","""BasicChecking""","""13648128567648…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
0.983033,"""83006559117044…","""13659211107211…","""FirstMortgage3…","""16866692171742…","""Rejected"""
0.144221,"""67209219753881…","""78351038508354…","""FirstMortgage3…","""97177809676306…","""Accepted"""
0.679745,"""17109820992353…","""18440160215443…","""MoneyMarketSav…","""16866692171742…","""Rejected"""
1.0,"""50329527454137…","""78351038508354…","""BasicChecking""","""16866692171742…","""Rejected"""
0.718982,"""49820764071531…","""13659211107211…","""BasicChecking""","""90736705467068…","""Accepted"""
0.0,"""16432312091900…","""18440160215443…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""22167054430828…","""18440160215443…","""BasicChecking""","""90736705467068…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""c73023dda51de1…","""../../../../da…","""af486bc1815c72…","""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
0.144221,"""5414b197fb87b3…","""../../../../da…","""3cdf7b1ae1342e…","""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
0.679745,"""892f5d61a9215e…","""../../../../da…","""806ef34eb8237f…","""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
1.0,"""c3aa807d9e927a…","""../../../../da…","""3cdf7b1ae1342e…","""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
0.718982,"""c2d492f9a05073…","""../../../../da…","""af486bc1815c72…","""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
0.0,"""b21059221605c4…","""../../../../da…","""806ef34eb8237f…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""80cbe489dcecd4…","""../../../../da…","""806ef34eb8237f…","""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
